In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import AveragePooling2D
from keras.layers import MaxPooling2D
from keras.layers import ZeroPadding2D

from keras.layers import Flatten
from keras.layers import Reshape

from keras.layers import UpSampling2D
from keras.layers import Conv2DTranspose

from keras.activations import relu

Using TensorFlow backend.


In [2]:
import os
from scipy.misc import imread
import cv2
import numpy as np

def img2mat(path = '..'):
    img = []
    
    for file in os.listdir():
        if file.endswith('.jpg'):
            origin_size_img = imread(file)
            new_size_img = cv2.resize(origin_size_img , (224,224) , interpolation = cv2.INTER_NEAREST)
        
            img.append(new_size_img)

    return np.array(img)

In [3]:
model = Sequential()

#encoder step
model.add(Conv2D(filters=64 , kernel_size=(3,3) , activation=relu , padding='same' , strides=(1,1) , input_shape=(224,224,3) , name='conv1'))
model.add(Conv2D(filters=64 , kernel_size=(3,3) , activation=relu , padding='same' , strides=(1,1) , name='conv2'))
model.add(MaxPooling2D(pool_size=(2,2) , padding='same' , name='pool1'))

model.add(Conv2D(filters=128 , kernel_size=(3,3) , activation=relu , padding='same' , strides=(1,1) , name='conv3'))
model.add(Conv2D(filters=128 , kernel_size=(3,3) , activation=relu , padding='same' , strides=(1,1) , name='conv4'))
model.add(MaxPooling2D(pool_size=(2,2) , padding='same' , name='pool2'))

model.add(Conv2D(filters=256 , kernel_size=(3,3) , activation=relu , padding='same' , strides=(1,1) , name='conv5'))
model.add(Conv2D(filters=256 , kernel_size=(3,3) , activation=relu , padding='same' , strides=(1,1) , name='conv6'))
model.add(Conv2D(filters=256 , kernel_size=(3,3) , activation=relu , padding='same' , strides=(1,1) , name='conv7'))
model.add(MaxPooling2D(pool_size=(2,2) , padding='same' , name='pool3'))

model.add(Conv2D(filters=512 , kernel_size=(3,3) , activation=relu , padding='same' , strides=(1,1) , name='conv8'))
model.add(Conv2D(filters=512 , kernel_size=(3,3) , activation=relu , padding='same' , strides=(1,1) , name='conv9'))
model.add(Conv2D(filters=512 , kernel_size=(3,3) , activation=relu , padding='same' , strides=(1,1) , name='conv10'))
model.add(MaxPooling2D(pool_size=(2,2) , padding='same' , name='pool4'))

model.add(Conv2D(filters=512 , kernel_size=(3,3) , activation=relu , padding='same' , strides=(1,1) , name='conv11'))
model.add(Conv2D(filters=512 , kernel_size=(3,3) , activation=relu , padding='same' , strides=(1,1) , name='conv12'))
model.add(Conv2D(filters=512 , kernel_size=(3,3) , activation=relu , padding='same' , strides=(1,1) , name='conv13'))
model.add(MaxPooling2D(pool_size=(2,2) , padding='same' , name='pool5'))

model.add(Flatten())

model.add(Dense(units=4096 , activation=relu , name='fc1'))
model.add(Dense(units=4096 , activation=relu , name='fc2'))
model.add(Dense(units=1000 , activation=relu , name='fc3'))

#VGG16

In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2 (Conv2D)               (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv3 (Conv2D)               (None, 112, 112, 128)     73856     
_________________________________________________________________
conv4 (Conv2D)               (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)       0         
_________________________________________________________________
conv5 (Conv2D)               (None, 56, 56, 256)       295168    
__________

In [4]:
#decoder step
model.add(Dense(units=4096 , activation=relu , name='de-fc1'))
model.add(Dense(units=4096 , activation=relu , name='de-fc2'))
model.add(Dense(units=25088 , activation=relu , name='de-fc3'))

model.add(Reshape((7 , 7 , 512)))

model.add(UpSampling2D(size=(2,2) , name='upsamp1'))

model.add(Conv2DTranspose(filters=512 , kernel_size=(3,3) , activation=relu , strides=(1,1) , padding='same' , name='upconv1'))
model.add(Conv2DTranspose(filters=512 , kernel_size=(3,3) , activation=relu , strides=(1,1) , padding='same' , name='upconv2'))
model.add(Conv2DTranspose(filters=512 , kernel_size=(3,3) , activation=relu , strides=(1,1) , padding='same' , name='upconv3'))

model.add(UpSampling2D(size=(2,2) , name='upsamp2'))

model.add(Conv2DTranspose(filters=512 , kernel_size=(3,3) , activation=relu , strides=(1,1) , padding='same' , name='upconv4'))
model.add(Conv2DTranspose(filters=512 , kernel_size=(3,3) , activation=relu , strides=(1,1) , padding='same' , name='upconv5'))
model.add(Conv2DTranspose(filters=512 , kernel_size=(3,3) , activation=relu , strides=(1,1) , padding='same' , name='upconv6'))

model.add(UpSampling2D(size=(2,2) , name='upsamp3'))

model.add(Conv2DTranspose(filters=256 , kernel_size=(3,3) , activation=relu , strides=(1,1) , padding='same' , name='upconv7'))
model.add(Conv2DTranspose(filters=256 , kernel_size=(3,3) , activation=relu , strides=(1,1) , padding='same' , name='upconv8'))
model.add(Conv2DTranspose(filters=256 , kernel_size=(3,3) , activation=relu , strides=(1,1) , padding='same' , name='upconv9'))

model.add(UpSampling2D(size=(2,2) , name='upsamp4'))

model.add(Conv2DTranspose(filters=128 , kernel_size=(3,3) , activation=relu , strides=(1,1) , padding='same' , name='upconv10'))
model.add(Conv2DTranspose(filters=128 , kernel_size=(3,3) , activation=relu , strides=(1,1) , padding='same' , name='upconv11'))

model.add(UpSampling2D(size=(2,2) , name='upsamp5'))

model.add(Conv2DTranspose(filters=64 , kernel_size=(3,3) , activation=relu , strides=(1,1) , padding='same' , name='upconv12'))
model.add(Conv2DTranspose(filters=64 , kernel_size=(3,3) , activation=relu , strides=(1,1) , padding='same' , name='upconv13'))

#original image
model.add(Conv2DTranspose(filters=3 , kernel_size=(3,3) , activation=relu , strides=(1,1) , padding='same' , name='upconv14'))


In [59]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2 (Conv2D)               (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv3 (Conv2D)               (None, 112, 112, 128)     73856     
_________________________________________________________________
conv4 (Conv2D)               (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)       0         
_________________________________________________________________
conv5 (Conv2D)               (None, 56, 56, 256)       295168    
__________

In [5]:
from keras.losses import mse
from keras.optimizers import Adam

In [6]:
model.compile(optimizer=Adam() , loss=mse)

#model.fit()

In [81]:
train_data = img2mat()

In [ ]:
model.fit(train_data , train_data , batch_size=1 , epochs=1)

Epoch 1/1


In [12]:
#autoencoder
#降维
from keras.models import Model

autoencoder = Model(inputs = model.input , outputs=model.get_layer(name='fc3').output)

In [13]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1_input (InputLayer)     (None, 224, 224, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2 (Conv2D)               (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv3 (Conv2D)               (None, 112, 112, 128)     73856     
_________________________________________________________________
conv4 (Conv2D)               (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)       0         
__________

In [ ]:
#对新数据进行降维
#可以自行指定维数
new_data = img2mat(path='new_data_path')

new_features = autoencoder.predict(new_data)


In [18]:
autodecoder = Model(inputs=model.get_layer(name='de-fc1').input , outputs=model.output)

I:\Anaconda3\lib\site-packages\keras\engine\topology.py:1574: UserWarning: Model inputs must come from a Keras Input layer, they cannot be the output of a previous non-Input layer. Here, a tensor specified as input to "model_4" was not an Input tensor, it was generated by layer fc3.
Note that input tensors are instantiated via `tensor = Input(shape)`.
The tensor that caused the issue was: fc3/Relu:0
  str(x.name))


TypeError: Input layers to a `Model` must be `InputLayer` objects. Received inputs: Tensor("fc3/Relu:0", shape=(?, 1000), dtype=float32). Input 0 (0-based) originates from layer type `Dense`.

In [20]:
model.get_layer(name='de-fc1').input

<tf.Tensor 'fc3/Relu:0' shape=(?, 1000) dtype=float32>